# Solution to Python Mentorship Assignment

## Part 1:

### Q1. Mutable Default Arguments

**Soln:**

- Default argument values in Python are evaluated **once** at function *definition* time, not each time the function is called.
- So `box=[]` creates **one single list object** when the function is defined, and that same list is reused for every call where `box` is not provided.
- Every time you call `add_item(x)` without passing a `box`, the item is appended to that same shared list.

**Why does `box` persist data between function calls?**

- Because `box` points to the same list object for all calls that omit the second argument.
- The list lives in the function object’s `__defaults__` and is **not recreated** on each call. So previous items stay there.

**Corrected function header (standard fix):**

```python
def add_item(item, box=None):
    if box is None:  # create a new list for each call that doesn't pass a box
        box = []
    box.append(item)
    return box
```

Using `None` as the default and creating a new list *inside* the function avoids the “shared list” bug.

### Q2. `__str__` vs `__repr__`

**The difference is as follows**

- `__repr__`:
  - Aimed at **developers**.
  - Should be an **unambiguous** representation of the object.
  - Ideally, it looks like valid Python code that could recreate the object (this is a convention, not a strict requirement).

- `__str__`:
  - Aimed at **end users / humans**.
  - Should be a **readable, nicely formatted** description of the object.

**Fallback behavior:**

- `str(obj)` (and `print(obj)`) call `obj.__str__()`.
- If `__str__` is **not** defined, Python falls back to `__repr__`.
- `repr(obj)` always calls `__repr__` (or a default from `object` if not defined).

### Q3. Class Variables vs. Instance Variables

**Memory difference:**

- **Instance variable** (defined in `__init__` using `self.var`):
  - Stored in each object’s own `__dict__`.
  - Every instance gets its **own copy** of that attribute.
  - Memory scales with the number of instances.

- **Class variable** (defined directly under `class Name:`):
  - Stored in the class’s namespace, **shared** by all instances that don’t override it.
  - Only **one copy** exists at the class level.


- If we do `ClassName.var = new_value`:
  - We change the single class-level attribute.
  - All instances that have **not** set their own `instance.var` will see the new value.

- If we do `instance.var = new_value`:
  - Python **creates or updates an instance attribute** named `var` on that specific object.
  - This **shadows** (overrides) the class variable *for that instance only*.
  - The class variable on `ClassName.var` and other instances that haven’t overridden it are unaffected.

## Part 2: Programming Challenges

### Q4. Complex Dictionary Parsing (Log Analysis)

We parse the log string and track whether each user is currently **Online** or **Offline**.
Everyone starts as Offline. Consecutive logins/logouts simply keep them in the correct state.

In [ ]:
def parse_user_logs(log_string: str) -> dict:
    """Parse a server log string and return current online/offline state per user.

    Assumptions:
    - Everyone starts as 'Offline'.
    - 'Login' -> set state to 'Online'.
    - 'Logout' -> set state to 'Offline'.
    - Consecutive logins/logouts are handled gracefully by re-setting the state.
    """
    states = {}  # user -> 'Online' / 'Offline'

    # Split by ';' to get individual events
    events = log_string.split(';')
    for event in events:
        event = event.strip()
        if not event:
            continue  # ignore empty segments

        # Expect format: 'UserX: Action'
        try:
            user_part, action_part = event.split(':', maxsplit=1)
        except ValueError:
            # Malformed line, skip or handle as needed
            continue

        user = user_part.strip()
        action = action_part.strip().lower()  # make it case-insensitive

        # Ensure user has a default state
        states.setdefault(user, 'Offline')

        if 'login' in action:
            states[user] = 'Online'
        elif 'logout' in action:
            states[user] = 'Offline'
        # else: ignore unknown actions

    return states


### Q5. The "Safe" Calculator (Error Handling)

This function repeatedly asks the user for two numbers and an operator,
handles `ZeroDivisionError` and `ValueError`, prints the result in an `else` block,
and always prints a message in `finally`.

In [ ]:
def safe_calculator():
    """A safe calculator that handles errors and uses else/finally blocks.

    - Repeatedly asks the user for two numbers and an operator.
    - Handles ZeroDivisionError and ValueError explicitly.
    - Uses 'else' to print the result only if no error occurred.
    - Uses 'finally' to print a message regardless of outcome.
    - Asks user whether to continue after each attempt.
    """
    while True:
        try:
            first = float(input("Enter the first number: "))
            second = float(input("Enter the second number: "))
            op = input("Enter operator (+, -, *, /): ").strip()

            if op == '+':
                result = first + second
            elif op == '-':
                result = first - second
            elif op == '*':
                result = first * second
            elif op == '/':
                # This can raise ZeroDivisionError
                result = first / second
            else:
                print("Unsupported operator. Please use +, -, *, or /.")
                continue  

        except ZeroDivisionError:
            print("Cannot divide by zero")
        except ValueError:
            print("Invalid input. Please enter numeric values for the numbers.")
        else:
            print("Result:", result)
        finally:
            print("Execution attempt complete")

        cont = input("Do you want to perform another calculation? (y/n): ").strip().lower()
        if cont != 'y':
            print("Exiting calculator.")
            break

## Part 3: Advanced OOP Challenges

### Q6. Class Interaction & State Management (Library System)

We create `Book` and `Library` classes. The `Library` modifies the state
of `Book` instances stored in its internal list.

In [ ]:
class Book:
    def __init__(self, title: str, author: str):
        self.title = title
        self.author = author
        self.is_checked_out = False

    def __repr__(self):
        status = 'Checked out' if self.is_checked_out else 'Available'
        return f"Book(title={self.title!r}, author={self.author!r}, status={status!r})"


class Library:
    def __init__(self):
        self.books = []  # list of Book objects

    def add_book(self, book_obj: Book) -> None:
        """Add a Book object to the library."""
        self.books.append(book_obj)

    def _find_book_by_title(self, title: str) -> Book | None:
        for book in self.books:
            if book.title == title:
                return book
        return None

    def checkout_book(self, title: str) -> None:
        """Check out a book by title.

        - If the book is found and available, mark it as checked out.
        - If already checked out, raise a RuntimeError.
        - If not found, raise a ValueError.
        """
        book = self._find_book_by_title(title)
        if book is None:
            raise ValueError(f"Book with title {title!r} not found in library.")
        if book.is_checked_out:
            raise RuntimeError(f"Book {title!r} is already checked out.")
        book.is_checked_out = True
        print(f"Checked out: {book.title}")

    def return_book(self, title: str) -> None:
        """Return a book by title.

        - If the book is found, mark it as not checked out.
        - If the book was not checked out, just inform the user.
        - If not found, raise a ValueError.
        """
        book = self._find_book_by_title(title)
        if book is None:
            raise ValueError(f"Book with title {title!r} not found in library.")
        if not book.is_checked_out:
            print(f"Book {title!r} was not checked out.")
        else:
            book.is_checked_out = False
            print(f"Returned: {book.title}")


# Demonstration:
library = Library()
book1 = Book("1984", "George Orwell")
book2 = Book("Brave New World", "Aldous Huxley")

library.add_book(book1)
library.add_book(book2)

print("Initial library state:")
print(library.books)

library.checkout_book("1984")
print("After checking out '1984':")
print(library.books)

library.return_book("1984")
print("After returning '1984':")
print(library.books)

### Q7. Encapsulation with Property Decorators

- `email` is computed dynamically via `@property`.
- `salary` uses a setter that disallows negative values.
- `fullname` has a deleter that sets `first` and `last` to `None`.

In [ ]:
class Employee:
    def __init__(self, first: str, last: str, salary: float):
        self.first = first
        self.last = last
        self._salary = salary  # internal storage for salary

    @property
    def email(self) -> str | None:
        """Construct email dynamically from first and last.

        If first/last are None, return None.
        """
        if self.first is None or self.last is None:
            return None
        return f"{self.first.lower()}.{self.last.lower()}@company.com"

    @property
    def salary(self) -> float:
        return self._salary

    @salary.setter
    def salary(self, value: float) -> None:
        if value < 0:
            raise ValueError("Salary cannot be negative")
        self._salary = value

    @property
    def fullname(self) -> str | None:
        if self.first is None and self.last is None:
            return None
        return f"{self.first} {self.last}"

    @fullname.deleter
    def fullname(self) -> None:
        """Delete the fullname by setting first and last to None."""
        self.first = None
        self.last = None

    def __repr__(self):
        return f"Employee(first={self.first!r}, last={self.last!r}, salary={self._salary!r})"


### Q8. Operator Overloading (`TimeDuration`)

The `TimeDuration` class:
- Normalizes hours and minutes in `__init__`.
- Overloads `+` via `__add__` to add two durations.
- Implements `__str__` to print in the form `XH:YM`.

In [ ]:
class TimeDuration:
    def __init__(self, hours: int = 0, minutes: int = 0):
        # Convert everything to total minutes, then normalize
        total_minutes = hours * 60 + minutes
        if total_minutes < 0:
            raise ValueError("TimeDuration cannot represent negative time in this implementation")

        self.hours = total_minutes // 60
        self.minutes = total_minutes % 60

    def __add__(self, other: "TimeDuration") -> "TimeDuration":
        if not isinstance(other, TimeDuration):
            return NotImplemented
        # Adding via total minutes automatically handles minute rollover
        total_minutes = self.hours * 60 + self.minutes + other.hours * 60 + other.minutes
        return TimeDuration(minutes=total_minutes)

    def __str__(self) -> str:
        return f"{self.hours}H:{self.minutes}M"

    def __repr__(self) -> str:
        return f"TimeDuration(hours={self.hours}, minutes={self.minutes})"


# Test example from the assignment
t1 = TimeDuration(hours=2, minutes=45)
t2 = TimeDuration(hours=1, minutes=30)
t3 = t1 + t2

print("t1 =", t1)
print("t2 =", t2)
print("t1 + t2 =", t3)  # expected 4H:15M